In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump, symb_perf_stats_vectorized

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AAIN', 'AAMC', 'ABMD', 'ABST', 'AC', 'ACAB', 'ACAD', 'ACAQ', 'ACBA',
       'ACDI',
       ...
       'WRAC', 'WSC', 'WTMA', 'WWAC', 'XEL', 'XFIN', 'XPEL', 'XPER', 'YNDX',
       'ZNH'],
      dtype='object', length=731)


In [18]:
df_v.head(2)
df_v.tail(2)

,A,AA,AADI,AAIN,AAL,AAMC,AAN,AAON,AAP,AAPL,...,ZIP,ZLAB,ZM,ZNH,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-06,2444600.0,4637800,205500.0,0.0,29525900.0,18200.0,237400.0,207000.0,717900.0,87686600.0,...,410100.0,2092500.0,4078300.0,7400.0,611700.0,2612800.0,1855300.0,1761200.0,338900.0,568500.0
2023-01-09,1194393.0,6084353,89911.0,1003.0,31363255.0,4253.0,307813.0,184041.0,743188.0,70138063.0,...,654434.0,1024013.0,3401503.0,17091.0,886430.0,2957624.0,2393006.0,1700125.0,587887.0,580242.0


In [19]:
df_s = df_v.shift(periods=1)
df_s.head(2)
df_s.tail(2)

,A,AA,AADI,AAIN,AAL,AAMC,AAN,AAON,AAP,AAPL,...,ZIP,ZLAB,ZM,ZNH,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-06,1714600.0,3835700.0,49300.0,0.0,27378000.0,7000.0,211200.0,168300.0,795700.0,80962700.0,...,382500.0,5599800.0,2040100.0,11200.0,960800.0,3931100.0,2803200.0,1560900.0,388800.0,1213300.0
2023-01-09,2444600.0,4637800.0,205500.0,0.0,29525900.0,18200.0,237400.0,207000.0,717900.0,87686600.0,...,410100.0,2092500.0,4078300.0,7400.0,611700.0,2612800.0,1855300.0,1761200.0,338900.0,568500.0


In [22]:
dif = df_v - df_s
dif.head(2)
dif.tail(2)

,A,AA,AADI,AAIN,AAL,AAMC,AAN,AAON,AAP,AAPL,...,ZIP,ZLAB,ZM,ZNH,ZNTL,ZS,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-06,730000.0,802100.0,156200.0,0.0,2147900.0,11200.0,26200.0,38700.0,-77800.0,6723900.0,...,27600.0,-3507300.0,2038200.0,-3800.0,-349100.0,-1318300.0,-947900.0,200300.0,-49900.0,-644800.0
2023-01-09,-1250207.0,1446553.0,-115589.0,1003.0,1837355.0,-13947.0,70413.0,-22959.0,25288.0,-17548537.0,...,244334.0,-1068487.0,-676797.0,9691.0,274730.0,344824.0,537706.0,-61075.0,248987.0,11742.0


In [27]:
rows, cols = np.where(dif == 0)
print(len(rows), rows)
print(len(cols), cols)
print(len(df_v)) 

44601 [   1    1    1 ... 1498 1498 1498]
44601 [ 121  170  175 ... 3532 3534 3611]
1500


In [30]:
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
len(idx_same_volume)

1859

In [35]:
df_v.iloc[1][122]

349000.0

In [5]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['AADI', 'AAIN', 'AAMC', 'AAN', 'ABCL', 'ABCM', 'ABMD', 'ABNB', 'ACA',
       'ACAB',
       ...
       'YOU', 'ZEN', 'ZGN', 'ZI', 'ZIM', 'ZIP', 'ZLAB', 'ZM', 'ZNTL', 'ZS'],
      dtype='object', length=1110)


In [6]:
symbols_drop = list(symbols_no_close) + list(symbols_no_volume)  # combine symbols with no volume and no close
print(f'combined symbols with no volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
symbols_drop .sort()
df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

combined symbols with no volume and no close, inculdes duplicate symbols: 1841
unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 1336


In [7]:
print(f'symbols with no volume: {len(symbols_no_volume)}')
print(f'symbols with no close: {len(symbols_no_close)}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop)}')
print(f'symbols, rows in df adjOHLCV (df_a) after dropped symbols: {len(df_a.columns)/5:.0f}, {len(df_a)}')
print(f'symbols, rows in df Close (df_c) after dropped symbols: {len(df_c.columns)}, {len(df_c)}')


symbols with no volume: 731
symbols with no close: 1110
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): 1336
symbols, rows in df adjOHLCV (df_a) after dropped symbols: 2302, 1500
symbols, rows in df Close (df_c) after dropped symbols: 2302, 1500


In [8]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean
